## Support Vector Machines




In [4]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sk2torch"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
import numpy as np
from sklearn.svm import SVC
import sk2torch
import torch
import ezkl
import os


# Create a dataset of two Gaussians. There will be some overlap
# between the two classes, which adds some uncertainty to the model.
xs = np.array([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
    [4.7, 3.2, 1.3, 0.2],
    [4.6, 3.1, 1.5, 0.2],
    [5.0, 3.6, 1.4, 0.2],
    [5.4, 3.9, 1.7, 0.4],
    [4.6, 3.4, 1.4, 0.3],
    [5.0, 3.4, 1.5, 0.2],
    [4.4, 2.9, 1.4, 0.2],
    [4.9, 3.1, 1.5, 0.1],
    [7.0, 3.2, 4.7, 1.4],
    [6.4, 3.2, 4.5, 1.5],
    [6.9, 3.1, 4.9, 1.5],
    [5.5, 2.3, 4.0, 1.3],
    [6.5, 2.8, 4.6, 1.5],
    [5.7, 2.8, 4.5, 1.3],
    [6.3, 3.3, 4.7, 1.6],
    [4.9, 2.4, 3.3, 1.0],
    [6.6, 2.9, 4.6, 1.3],
    [5.2, 2.7, 3.9, 1.4],
])
ys = np.array([
    -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
])

# Train an SVM on the data and wrap it in PyTorch.
sk_model = SVC(kernel="linear")
sk_model.fit(xs, ys)
# Extract the weights
W = sk_model.coef_

# Print the weights
print("Weights (W):", W)
model = sk2torch.wrap(sk_model)

# Convert xs to a PyTorch tensor
xs_tensor = torch.tensor(xs)

# Use the model to predict labels
y_hat = model.predict(xs_tensor)
print(y_hat)

Weights (W): [[ 0.08203157 -0.36939205  0.79921364  0.34779002]]
tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1])


In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
print(xs.shape)

In [ ]:


# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = xs.shape
torch_out = y_hat
# Export the model
torch.onnx.export(model,               # model being run
                  # model input (or a tuple for multiple inputs)
                  xs_tensor,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((xs_tensor).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[shape],
            input_data=[d],
            output_data=[o.reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))


In [ ]:
!RUST_LOG=trace
run_args = ezkl.PyRunArgs();
run_args.variables = [("batch_size", shape[0])]
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args) 
assert res == True

In [ ]:
ezkl.calibrate_settings(data_path, model_path, settings_path, "resources", scales=[3,7])

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = ezkl.get_srs(settings_path)

In [ ]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF
import time

proof_path = os.path.join('test.pf')
# log time it takes to generate proof
start = time.time()

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )
end = time.time()
proving_time = end - start
print("PROOF GENERATION TIME: ", proving_time)

# define the path that stores the benchmarking results
benchmark_path = os.path.join('../../benchmarks.json')

# check that a benchmark path exists. If not, create one. Otherwise, load the existing one
if not os.path.isfile(benchmark_path):
    data = {
        "svm_classifications": {
            "ezkl": {
                "provingTime": proving_time
            },
            "riscZero": {}
        }
    }
    with open(benchmark_path, 'w') as f:
        json.dump(data, open(benchmark_path, 'w'))
else:
    with open(benchmark_path, 'r') as f:
        benchmark = json.load(f)

    proving_time =str(proving_time) + "s"

    # Update the proving time in the loaded benchmark
    benchmark['svm_classifications']['ezkl']['provingTime'] = proving_time
    

    # Write the updated benchmark back to the file
    with open(benchmark_path, 'w') as f:
        json.dump(benchmark, f, indent=4)


print(res['instances'])
assert os.path.isfile(proof_path)